---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [5]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [6]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

In [7]:
for G in P1_Graphs:
    print(len(G.nodes()), nx.average_shortest_path_length(G), nx.average_clustering(G))

1000 4.099161161161161 0.03167539146454044
1000 5.089871871871872 0.5642419635919628
750 9.378702269692925 0.4018222222222227
750 3.1048046283934134 0.03780379975223251
750 5.0785509568313305 0.0033037037037037037


<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [8]:
def graph_identification():
    
    # Your Code Here
    
    return ['PA', 'SW_H', 'SW_L', 'SW_H', 'PA']# Your Answer Here

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [9]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [10]:
nodes = G.nodes(data=True)

In [11]:
df = {}
for node, attr in nodes:
    df[node] = {
                'department': attr['Department'],
                'management': attr['ManagementSalary'],
                'degree_centrality': nx.degree_centrality(G)[node],
#                 'c2': nx.closeness_centrality(G)[node],
#                 'c3': nx.betweenness_centrality(G, normalized=True, endpoints=False)[node]
               }

In [12]:
df = pd.DataFrame.from_dict(df, orient='index')

In [13]:
df['department'] = df['department'].astype(str)

In [14]:
X = pd.get_dummies(df[['department', 'degree_centrality']])
y = df['management']

In [24]:
X_train = X.loc[~y.isnull()]
X_test = X.loc[y.isnull()]
y_train = y.loc[~y.isnull()].astype(int)
y_test = y.loc[y.isnull()]

In [25]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((753, 43), (252, 43), (753,), (252,))

In [26]:
from sklearn.linear_model import LogisticRegression

In [ ]:
from sklearn.model_selection import cross_val_score

In [51]:
lm = LogisticRegression()

In [52]:
scores = cross_val_score(lm, X_train, y_train, cv=5, scoring='roc_auc')

In [53]:
scores

array([ 0.69914698,  0.7847769 ,  0.63090551,  0.59498031,  0.64044168])

In [ ]:
def salary_predictions():
    
    # Your Code Here
    
    return # Your Answer Here

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [ ]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [ ]:
def new_connections_predictions():
    
    # Your Code Here
    
    return # Your Answer Here